In [4]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1330k  100 1330k    0     0  1773k      0 --:--:-- --:--:-- --:--:-- 1771k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1253k  100 1253k    0     0  3634k      0 --:--:-- --:--:-- --:--:-- 3644k


In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [7]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [8]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [ ]:
df = df_train[['PULocationID', 'DOLocationID', "trip_distance", "duration"]]

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/zapatacc/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as zapatacc

Initialized MLflow to track repo "zapatacc/nyc-taxi-time-prediction"

Repository zapatacc/nyc-taxi-time-prediction initialized!

2024/09/17 21:27:42 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


https://dagshub.com/zapatacc/nyc-taxi-time-prediction.mlflow


<Experiment: artifact_location='mlflow-artifacts:/d9a287ef0aa644de8a92317afa69e25e', creation_time=1726630062277, experiment_id='0', last_update_time=1726630062277, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [10]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("zapatacc/nyc-taxi-time-prediction")

# Upload file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="eval_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [11]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:7.49280                                                                                                                          
[1]	validation-rmse:6.53786                                                                                                                          
[2]	validation-rmse:6.00633                                                                                                                          
[3]	validation-rmse:5.70562                                                                                                                          
[4]	validation-rmse:5.54147                                                                                                                          
[5]	validation-rmse:5.43053                                                                                                                          
[6]	validation-rmse:5.37692                                                                         

2024/09/17 21:39:12 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:39:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:39:12] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:39:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:39:19] WARNING: /Users/runner

[0]	validation-rmse:8.74750                                                                                                                          
[1]	validation-rmse:8.41690                                                                                                                          
[2]	validation-rmse:8.11708                                                                                                                          
[3]	validation-rmse:7.84595                                                                                                                          
[4]	validation-rmse:7.60095                                                                                                                          
[5]	validation-rmse:7.37924                                                                                                                          
[6]	validation-rmse:7.18059                                                                         

2024/09/17 21:39:32 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:39:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:39:32] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:39:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:39:36] WARNING: /Users/runner

[0]	validation-rmse:7.30183                                                                                                                          
[1]	validation-rmse:6.35496                                                                                                                          
[2]	validation-rmse:5.89144                                                                                                                          
[3]	validation-rmse:5.67193                                                                                                                          
[4]	validation-rmse:5.56809                                                                                                                          
[5]	validation-rmse:5.51468                                                                                                                          
[6]	validation-rmse:5.47726                                                                         

...

2024/09/17 21:43:07 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:43:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:43:07] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:43:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:43:12] WARNING: /Users/runner

[0]	validation-rmse:7.89346                                                                                                                          
[1]	validation-rmse:7.03506                                                                                                                          
[2]	validation-rmse:6.44265                                                                                                                          
[3]	validation-rmse:6.04643                                                                                                                          
[4]	validation-rmse:5.77699                                                                                                                          
[5]	validation-rmse:5.60456                                                                                                                          
[6]	validation-rmse:5.49199                                                                         

2024/09/17 21:43:32 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/17 21:43:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:43:32] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/17 21:43:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:43:37] WARNING: /Users/runner

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [04:52<00:00, 29.27s/trial, best loss: 5.162204519821762]
[0]	validation-rmse:7.89346
[1]	validation-rmse:7.03506
[2]	validation-rmse:6.44265
[3]	validation-rmse:6.04643
[4]	validation-rmse:5.77699
[5]	validation-rmse:5.60456
[6]	validation-rmse:5.49199
[7]	validation-rmse:5.40957
[8]	validation-rmse:5.35781
[9]	validation-rmse:5.32430
[10]	validation-rmse:5.29878
[11]	validation-rmse:5.27674
[12]	validation-rmse:5.26299
[13]	validation-rmse:5.25095
[14]	validation-rmse:5.24244
[15]	validation-rmse:5.23532
[16]	validation-rmse:5.22995
[17]	validation-rmse:5.22582
[18]	validation-rmse:5.22250
[19]	validation-rmse:5.21863
[20]	validation-rmse:5.21749
[21]	validation-rmse:5.21575
[22]	validation-rmse:5.21476
[23]	validation-rmse:5.21379
[24]	validation-rmse:5.21280
[25]	validation-rmse:5.21140
[26]	validation-rmse:5.20986
[27]	validation-rmse:5.20777
[28]	validation-rmse:5.20553
[29]	validation-rms

2024/09/17 21:43:55 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/17 21:43:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/cristianzapata/Documents/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:43:55] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/17 21:43:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/17 21:44:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimization at: https://dagshub.com/zapatacc/nyc-taxi-time-

In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Ingrese el run_id a93d8afdb1c74b0091098c628654693a


Successfully registered model 'nyc-taxi-model'.
2024/09/17 21:46:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [16]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726631219517, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-17 21:48:37.092155', last_updated_timestamp=1726631317441, name='nyc-taxi-model', run_id='a93d8afdb1c74b0091098c628654693a', run_link='', source='mlflow-artifacts:/d9a287ef0aa644de8a92317afa69e25e/a93d8afdb1c74b0091098c628654693a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [17]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([18.962858, 28.185919,  7.922001, ..., 39.79563 , 13.458225,
       19.66295 ], dtype=float32)